In [23]:
import numpy as np

dataset = "CSpace1"

training_file = "./data/%s_training.csv" % dataset
holdout_file = "./data/%s_validation.csv" % dataset
test_file = "./data/%s_test.csv" % dataset


training_data = np.loadtxt(training_file, skiprows=0, delimiter=',')
validation_data = np.loadtxt(holdout_file, skiprows=0, delimiter=',')
test_data = np.loadtxt(test_file, skiprows=0, delimiter=',')

training_X = training_data[:, 0:2]
training_y = training_data[:, 2]
training_y[training_y == -1] = 0 # convert y to 0;

validation_X = validation_data[:, 0:2]
validation_y = validation_data[:, 2]
validation_y[validation_y == -1] = 0

test_X = test_data[:, 0:2]
test_y = test_data[:, 2]
test_y[test_y == -1] = 0 # convert y to 0;

print(training_X.shape)
print(training_y.shape)

print(validation_X.shape)
print(validation_y.shape)

print(test_X.shape,)
print(test_y.shape,)

print(np.unique(training_y, return_counts=True))
print(np.unique(test_y, return_counts=True))
# print("Number of positive vs negative samples: %s vs %s" % (np.count_nonzero(training_y == 1), np.count_nonzero(training_y == -1)))

(600, 2)
(600,)
(200, 2)
(200,)
(200, 2)
(200,)
(array([0., 1.]), array([124, 476]))
(array([0., 1.]), array([ 49, 151]))


In [24]:
from scipy.spatial.distance import cdist

def rbf(x1, x2, g=1):
    return np.exp(-g*(cdist(x1, x2, 'euclidean')**2))

def testrbf():
    x1 = np.array([[0, 1], [0, 2]])
    x2 = np.array([[1, 0], [2, 0]])
    out = rbf(x1,x2, 1)
    expected = np.array([[np.exp(-2), np.exp(-5)], [np.exp(-5), np.exp(-8)]])
    assert np.allclose(out, expected), "RBF output are not equal!. Expected: %s; Actual: %s" % (expected, out)
    print("Test for rbf passed.")
    
testrbf()

g = 1;
K_train = rbf(training_X, training_X, g)
K_validation = rbf(validation_X, training_X)
K_test = rbf(test_X, training_X)

print("Shape of K for training, test, validation: (%s, %s, %s)" % (K_train.shape, K_validation.shape, K_test.shape))

Test for rbf passed.
Shape of K for training, test, validation: ((600, 600), (200, 600), (200, 600))


In [25]:
from sklearn.linear_model import SGDClassifier
import json

sgd_output_file = "./sgd_%s.json" % dataset

sgd_clf = SGDClassifier(loss='log', alpha=1e-4, random_state=0, verbose=1).fit(K_train, training_y)
y_lr_holdout = sgd_clf.predict(K_validation)
p_lr_holdout = sgd_clf.predict_proba(K_validation).T
y_lr_test = sgd_clf.predict(K_test)
p_lr_test = sgd_clf.predict_proba(K_test).T # Need to align the first axis with y_pred

header_sgd = ['y_pred'] + sgd_clf.classes_.tolist()
print(header_sgd)
output_sgd_validation = np.vstack((y_lr_holdout, p_lr_holdout)).T
output_sgd_test = np.vstack((y_lr_test, p_lr_test)).T
output_sgd = {
    'coef': sgd_clf.coef_.tolist(),
    'intercept': sgd_clf.intercept_.tolist(),
    'output_header': header_sgd,
    'validation_output': output_sgd_validation.tolist(),
    'test_output': output_sgd_test.tolist()
}

with open(sgd_output_file, 'w') as outfile:
    json.dump(output_sgd, outfile)

-- Epoch 1
Norm: 166.28, NNZs: 600, Bias: 56.882541, T: 600, Avg. loss: 11.468485
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 155.80, NNZs: 600, Bias: 47.830188, T: 1200, Avg. loss: 4.406536
Total training time: 0.01 seconds.
-- Epoch 3
Norm: 147.24, NNZs: 600, Bias: 43.322377, T: 1800, Avg. loss: 2.927773
Total training time: 0.01 seconds.
-- Epoch 4
Norm: 130.30, NNZs: 600, Bias: 48.185511, T: 2400, Avg. loss: 2.119621
Total training time: 0.01 seconds.
-- Epoch 5
Norm: 122.43, NNZs: 600, Bias: 45.754005, T: 3000, Avg. loss: 1.837614
Total training time: 0.01 seconds.
-- Epoch 6
Norm: 116.82, NNZs: 600, Bias: 44.924828, T: 3600, Avg. loss: 1.425079
Total training time: 0.01 seconds.
-- Epoch 7
Norm: 112.43, NNZs: 600, Bias: 43.897108, T: 4200, Avg. loss: 1.204609
Total training time: 0.02 seconds.
-- Epoch 8
Norm: 105.30, NNZs: 600, Bias: 40.691806, T: 4800, Avg. loss: 0.761550
Total training time: 0.02 seconds.
-- Epoch 9
Norm: 100.60, NNZs: 600, Bias: 39.916314, T: 5400, Av

In [26]:
from sklearn.neural_network import MLPClassifier

mlp_output_file = "./mlp_%s.json" % dataset

# TODO: Do we need to transform the output from {-1, 1} to {0, 1}?
mlp_clf = MLPClassifier(solver='adam', alpha=1e-4, activation='relu',hidden_layer_sizes=(256, 256),
                        random_state=1, max_iter=1000, verbose=True).fit(training_X, training_y)

y_mlp_validation = mlp_clf.predict(validation_X)
p_mlp_validation = mlp_clf.predict_proba(validation_X).T

y_mlp_test = mlp_clf.predict(test_X)
p_mlp_test = mlp_clf.predict_proba(test_X).T

header_mlp = ['y_pred'] + mlp_clf.classes_.tolist()
print(header_mlp)
output_mlp_validation = np.vstack((y_mlp_validation, p_mlp_validation)).T
output_mlp_test = np.vstack((y_mlp_test, p_mlp_test)).T

coefs = []
intercepts = []
for coef in mlp_clf.coefs_:
    coefs.append(coef.tolist())

for intercept in mlp_clf.intercepts_:
    intercepts.append(intercept.tolist())

output_mlp = {
    'coef': coefs,
    'intercept': intercepts,
    'output_header': header_mlp,
    'validation_output': output_mlp_validation.tolist(),
    'test_output': output_mlp_test.tolist()
}

with open(mlp_output_file, 'w') as outfile:
    json.dump(output_mlp, outfile)

Iteration 1, loss = 0.59542125
Iteration 2, loss = 0.48002085
Iteration 3, loss = 0.43638617
Iteration 4, loss = 0.42482495
Iteration 5, loss = 0.41758305
Iteration 6, loss = 0.41323227
Iteration 7, loss = 0.40841122
Iteration 8, loss = 0.40301937
Iteration 9, loss = 0.39635326
Iteration 10, loss = 0.39044360
Iteration 11, loss = 0.38482503
Iteration 12, loss = 0.37969294
Iteration 13, loss = 0.37663878
Iteration 14, loss = 0.37191823
Iteration 15, loss = 0.36789985
Iteration 16, loss = 0.36445140
Iteration 17, loss = 0.36051554
Iteration 18, loss = 0.35615383
Iteration 19, loss = 0.35256819
Iteration 20, loss = 0.34826245
Iteration 21, loss = 0.34414732
Iteration 22, loss = 0.34034957
Iteration 23, loss = 0.33707541
Iteration 24, loss = 0.33177779
Iteration 25, loss = 0.32757901
Iteration 26, loss = 0.32394447
Iteration 27, loss = 0.31977346
Iteration 28, loss = 0.31477472
Iteration 29, loss = 0.30986282
Iteration 30, loss = 0.30567983
Iteration 31, loss = 0.30084084
Iteration 32, los

In [27]:
for coef in mlp_clf.coefs_:
    print(coef.shape,)
    
for b in mlp_clf.intercepts_:
    print(b.shape,)

(2, 256)
(256, 256)
(256, 1)
(256,)
(256,)
(1,)
